#PIP INSTALL

In [ ]:
# !pip uninstall numpy -y
# !pip install numpy


In [ ]:
# pip install dill

In [ ]:
pip install import_ipynb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.7 MB/s eta 0:00:00


#IMPORT

In [ ]:
import numpy as np
import pandas as pd
import joblib
import dill
import torch
import math

In [ ]:
# !pip install import_ipynb
from google.colab import drive
drive.mount("/content/drive")

%cd /content/drive/MyDrive/Colab Notebooks

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


In [ ]:
import import_ipynb
from model import AdditionModel,SubtractionModel,MultiplicationModel,DivisionModel,FactorialModel,ExponentialModel,Power_Model,LnModel,SineModel,predict_sinh,ArcsinModel

add = AdditionModel()
add.load_model('/content/drive/MyDrive/Colab Notebooks/Trained models/addition.pkl')

sub = SubtractionModel()
sub.load_model('/content/drive/MyDrive/Colab Notebooks/Trained models/subtraction.pkl')

mul = MultiplicationModel()
mul.load_model('/content/drive/MyDrive/Colab Notebooks/Trained models/multiplication.pkl')

exp= ExponentialModel()
exp.load('/content/drive/MyDrive/Colab Notebooks/Trained models/exponential.pkl')

pow = Power_Model()
pow.load_model('/content/drive/MyDrive/Colab Notebooks/Trained models/power_model.pkl')

ln = LnModel()
saved = joblib.load("/content/drive/MyDrive/Colab Notebooks/Trained models/ln.pkl")
ln.model = saved['model']
ln.poly = saved['poly']

fact= FactorialModel()
fact.load('/content/drive/MyDrive/Colab Notebooks/Trained models/factorial_model.pkl')

sin = SineModel(data_path=None)
sin.load_model("/content/drive/MyDrive/Colab Notebooks/Trained models/sine_model.pkl")

data = joblib.load("/content/drive/MyDrive/Colab Notebooks/Trained models/arcsin_model.pkl")
model = data['model']
poly = data['poly']

def predict_arcsin(x_val):
    x_val = max(-1, min(1, x_val))  # Clamp to valid domain
    x_val_poly = poly.transform(np.array([[x_val]]))
    return model.predict(x_val_poly)[0]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [79]:
import math






# Precedence and associativity
precedence = {
    '+': 1, '-': 1,
    '*': 2, '/': 2,
    '^': 3, '!': 4
}

right_associative = {'^'}

# Supported functions and constants
functions = {
    'sin': math.sin,
    'cos': math.cos,
    'tan': math.tan,
    'cot': lambda x: 1 / math.tan(x),
    'sec': lambda x: 1 / math.cos(x),
    'csc': lambda x: 1 / math.sin(x),
    'log2': math.log2,
    'log10': math.log10,
    'ln': math.log,
    'sinh': math.sinh,
    'cosh': math.cosh,
    'tanh': math.tanh,
    'coth': lambda x: 1 / math.tanh(x),
    'sech': lambda x: 1 / math.cosh(x),
    'csch': lambda x: 1 / math.sinh,
    'arcsin': math.asin,
    'arccos': math.acos,
    'arctan': math.atan,
    'arccot': lambda x: math.pi / 2 - math.atan(x),
    'arcsec': lambda x: math.pi / 2 - math.asin(x),
    'arccsc': lambda x: math.pi / 2 - math.acos(x),
    'arcsinh': lambda x: math.log(x + math.sqrt(x**2 + 1)),
    'arccosh': lambda x: math.log(x + math.sqrt(x**2 - 1)),
    'arctanh': lambda x: 0.5 * math.log((1 + x) / (1 - x)),
    'arccoth': lambda x: 0.5 * math.log((math.tanh(x) + 1) / (math.tanh(x) - 1)),
    'arcsech': lambda x: 0.5 * math.log((math.cosh(x) + math.sinh(x)) / (math.cosh(x) - math.sinh(x))),
    'arccsch': lambda x: 0.5 * math.log((math.sinh(x) + math.cosh(x)) / (math.sinh(x) - math.cosh(x)))
}

constants = {
    'e': math.e,
    'pi': math.pi
}

# 1️⃣ Tokenize input
def tokenize(expr):
    tokens = []
    i = 0
    n = len(expr)

    while i < n:
        if expr[i].isspace():
            i += 1
        elif expr[i] in '+-' and (i == 0 or expr[i - 1] in '()+-*/^'):
            # Lookahead to determine if followed by a number
            sign = expr[i]
            lookahead = i + 1
            if lookahead < n and (expr[lookahead].isdigit() or expr[lookahead] == '.'):
                # It's a unary +/-
                i += 1
                num_str = sign
                decimal_seen = False
                while i < n and (expr[i].isdigit() or (expr[i] == '.' and not decimal_seen)):
                    if expr[i] == '.':
                        decimal_seen = True
                    num_str += expr[i]
                    i += 1
                tokens.append(float(num_str))
                continue
            else:
                # It's a standalone '+' or '-', not part of a number
                tokens.append(sign)
                i += 1
        elif expr[i].isdigit() or expr[i] == '.':
            num = ''
            while i < n and (expr[i].isdigit() or expr[i] == '.'):
                num += expr[i]
                i += 1
            tokens.append(float(num))
        elif expr[i].isalpha():
            func = ''
            while i < n and expr[i].isalpha():
                func += expr[i]
                i += 1
            if i < n and expr[i].isdigit():  # handles log2, log10
                while i < n and expr[i].isdigit():
                    func += expr[i]
                    i += 1
            tokens.append(func)
        elif expr[i] in '+-*/^!()':
            tokens.append(expr[i])
            i += 1
        else:
            raise ValueError(f"Invalid character: {expr[i]}")

    return tokens





# 2️⃣ Convert Infix to Postfix (Shunting Yard)
def to_postfix(tokens):
    output = []
    stack = []
    for token in tokens:
        if isinstance(token, (int,float)):
            output.append(token)
        elif token in constants:
            output.append(constants[token])
        elif token in functions:
            stack.append(token)
        elif token == '(':
            stack.append(token)
        elif token == ')':
            while stack and stack[-1] != '(':
                output.append(stack.pop())
            stack.pop()  # remove '('
            if stack and stack[-1] in functions:
                output.append(stack.pop())
        elif token in precedence:
            while (stack and stack[-1] in precedence and
                   (precedence[token] < precedence[stack[-1]] or
                   (precedence[token] == precedence[stack[-1]] and token not in right_associative))):
                output.append(stack.pop())
            stack.append(token)
    while stack:
        output.append(stack.pop())
    return output

# 3️⃣ Evaluate Postfix Expression
def evaluate_postfix(postfix):
    stack = []
    for token in postfix:
        if isinstance(token, (int,float)):
            stack.append(token)
        elif callable(token):  # function
            val = stack.pop()
            stack.append(token(val))
        elif token in ('+', '-', '*', '/', '^'):
            b = stack.pop()
            a = stack.pop()
            if token == '+': stack.append(a + b)
            elif token == '-': stack.append(a - b)
            elif token == '*': stack.append(a * b)
            elif token == '/': stack.append(a / b)
            elif token == '^': stack.append(a ** b)
        elif token=='!':
            val = stack.pop()
            stack.append(math.factorial(int(val)))
        elif token in functions:
            val = stack.pop()
            stack.append(functions[token](val))

        else:
            raise ValueError(f"Invalid token: {token}")

    return stack[0]

# 🚀 Master Function
def evaluate_actual_expression(expr):
    tokens = tokenize(expr)

    postfix = to_postfix(tokens)
    print(postfix)
    result = evaluate_postfix(postfix)
    return result

print(evaluate_actual_expression("8-(+9)"))

[8.0, 9.0, '-']
-1.0


In [82]:
# Precedence and associativity
precedence = {
    '+': 1, '-': 1,
    '*': 2, '/': 2,
    '^': 3, '!': 4
}

right_associative = {'^'}

# Supported functions and constants
sinh_predict = lambda x: predict_sinh(x, exp)
cosh_predict = lambda x: (exp.predict_exp(x) + exp.predict_exp(-x)) / 2
tanh_predict = lambda x: (exp.predict_exp(x) - exp.predict_exp(-x)) / (exp.predict_exp(x) + exp.predict_exp(-x))
epsilon = 1e-8

pred_functions = {
    'sin': sin.predict,
    'cos': lambda x: pow.predict_power(np.clip(1 - pow.predict_power(sin.predict(x), 2), 0, None), 0.5),
    'tan': lambda x: (s := sin.predict(x)) / pow.predict_power(np.clip(1 - pow.predict_power(s, 2), 0, None), 0.5),
    'cot': lambda x: pow.predict_power(np.clip(1 - pow.predict_power(s := sin.predict(x), 2), 0, None), 0.5) / s,
    'sec': lambda x: 1 / pow.predict_power(np.clip(1 - pow.predict_power(sin.predict(x), 2), 0, None), 0.5),
    'csc': lambda x: 1 / sin.predict(x),
    'log2': lambda x: ln.predict_ln(x) / 0.693147,
    'log10': lambda x: 0.4342944994430173044643303070245 * ln.predict_ln(x),
    'ln': ln.predict_ln,
    'sinh': lambda x:predict_sinh(x,exp),
    'cosh': lambda x:(exp.predict_exp(x)+ exp.predict_exp(-x))/2,
    'tanh': lambda x: (exp.predict_exp(x)- exp.predict_exp(x))/(exp.predict_exp(x)+ exp.predict_exp(x),-2),
    'coth': lambda x: (exp.predict_exp(x) + exp.predict_exp(x)) /
                      (exp.predict_exp(x) - exp.predict_exp(x),- 2),

    'sech': lambda x: 2 / (exp.predict_exp(x) + exp.predict_exp(x)),

    'csch': lambda x: 2 / (exp.predict_exp(x) - exp.predict_exp(x)),
    'arcsin': predict_arcsin,
    'arccos': lambda x:predict_arcsin(pow.predict_power(1-pow.predict_power(x,2),0.5)),
    'arctan': lambda x: (x/predict_arcsin(pow.predict_power(1+pow.predict_power(x,2),0.5))),
    'arccot': lambda x: math.pi / 2 - (x/predict_arcsin(pow.predict_power(1+pow.predict_power(x,2),0.5))),
    'arcsec': lambda x: math.pi / 2 - predict_arcsin(x),
    'arccsc': lambda x: math.pi / 2 - predict_arcsin(pow.predict_power(1-pow.predict_power(x,2),0.5)),
    'arcsinh': lambda x: ln.predict_ln(x + pow.predict_power(pow.predict_power(x, 2) + 1, 0.5)),
    'arccosh': lambda x: ln.predict_ln(x + pow.predict_power(x * x - 1, 0.5)),
    'arctanh': lambda x: mul.predict(ln.predict_ln((1 + x) / (1 - x)),0.5),
    'arccoth': lambda x: mul.predict(ln.predict_ln((tanh_predict(x) + 1) / (tanh_predict(x) - 1)),0.5),
    'arcsech': lambda x: mul.predict(ln.predict_ln((cosh_predict(x) + sinh_predict(x)) / (cosh_predict(x) - sinh_predict(x) + epsilon)),0.5),
    'arccsch': lambda x: mul.predict(ln.predict_ln((sinh_predict(x) + cosh_predict(x)) / (sinh_predict(x) - cosh_predict(x) + epsilon)),0.5)

}

constants = {
    'e': exp.predict_exp(1),
    'pi': math.pi
}

# 1️⃣ Tokenize input
def tokenize(expr):
    tokens = []
    i = 0
    n = len(expr)

    while i < n:
        if expr[i].isspace():
            i += 1
        elif expr[i] in '+-' and (i == 0 or expr[i - 1] in '()+-*/^'):
            # Lookahead to determine if followed by a number
            sign = expr[i]
            lookahead = i + 1
            if lookahead < n and (expr[lookahead].isdigit() or expr[lookahead] == '.'):
                # It's a unary +/-
                i += 1
                num_str = sign
                decimal_seen = False
                while i < n and (expr[i].isdigit() or (expr[i] == '.' and not decimal_seen)):
                    if expr[i] == '.':
                        decimal_seen = True
                    num_str += expr[i]
                    i += 1
                tokens.append(float(num_str))
                continue
            else:
                # It's a standalone '+' or '-', not part of a number
                tokens.append(sign)
                i += 1
        elif expr[i].isdigit() or expr[i] == '.':
            num = ''
            while i < n and (expr[i].isdigit() or expr[i] == '.'):
                num += expr[i]
                i += 1
            tokens.append(float(num))
        elif expr[i].isalpha():
            func = ''
            while i < n and expr[i].isalpha():
                func += expr[i]
                i += 1
            if i < n and expr[i].isdigit():  # handles log2, log10
                while i < n and expr[i].isdigit():
                    func += expr[i]
                    i += 1
            tokens.append(func)
        elif expr[i] in '+-*/^!()':
            tokens.append(expr[i])
            i += 1
        else:
            raise ValueError(f"Invalid character: {expr[i]}")

    return tokens




# 2️⃣ Convert Infix to Postfix (Shunting Yard)
def to_postfix(tokens):
    output = []
    stack = []
    for token in tokens:
        if isinstance(token, (int,float)):
            output.append(token)
        elif token in constants:
            output.append(constants[token])
        elif token in pred_functions:
            stack.append(token)
        elif token == '(':
            stack.append(token)
        elif token == ')':
            while stack and stack[-1] != '(':
                output.append(stack.pop())
            stack.pop()  # remove '('
            if stack and stack[-1] in pred_functions:
                output.append(stack.pop())
        elif token in precedence:
            while (stack and stack[-1] in precedence and
                   (precedence[token] < precedence[stack[-1]] or
                   (precedence[token] == precedence[stack[-1]] and token not in right_associative))):
                output.append(stack.pop())
            stack.append(token)
    while stack:
        output.append(stack.pop())
    return output

# 3️⃣ Evaluate Postfix Expression
def evaluate_postfix(postfix):
    stack = []
    for token in postfix:
        if isinstance(token, (int,float)):
            stack.append(token)
        elif callable(token):  # function
            val = stack.pop()
            stack.append(token(val))
        elif token in ('+', '-', '*', '/', '^'):
            b = stack.pop()
            a = stack.pop()
            if token == '+': stack.append(add.predict(pd.DataFrame([[a, b]]))[0])
            elif token == '-': stack.append(sub.predict(pd.DataFrame([[a, b]]))[0])
            elif token == '*': stack.append(mul.predict(a, b))
            elif token == '/': stack.append(mul.predict(a, 1/b))
            elif token == '^': stack.append(pow.predict_power(a, b))
        elif token=='!':
            val = stack.pop()
            stack.append(fact.predict_factorial(int(val)))
        elif token in pred_functions:
            val = stack.pop()
            stack.append(pred_functions[token](val))
        else:
            raise ValueError(f"Invalid token: {token}")

    return stack[0]

# 🚀 Master Function
def evaluate_expression(expr):
    tokens = tokenize(expr)

    postfix = to_postfix(tokens)
    # print(postfix)
    result = evaluate_postfix(postfix)
    return result

k=input("Enter the expression: ")
a=evaluate_actual_expression(k)
b=evaluate_expression(k)
print("actual:",round(a,6))
print("pred:",round(b,6))
print('accuracy:',(a-b)/a)


Enter the expression: 4*5
[4.0, 5.0, '*']
actual: 20.0
pred: 20.0
accuracy: 0.0
